In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
### Data
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.datasets import load_svmlight_file
import lightgbm
import pickle
from get_exp_new import lime_exp, shap_exp, random_exp, grad_exp
from lirme_v2 import LIRME
import matplotlib.pyplot as plt

In [5]:
### TODO: Get top 20% important feature and retrain :D 

In [10]:
dataset_name = 'web10k'

In [11]:
data_path = {
 'web10k': {
     'train': '/home/amir/code/ltr_document_lvl_explainability/data/web10k/Fold1/train.txt',
     'valid': '/home/amir/code/ltr_document_lvl_explainability/data/web10k/Fold1/vali.txt',
     'test': '/home/amir/code/ltr_document_lvl_explainability/data/web10k/Fold1/test.txt'
 },
 'yahoo': {
     'train': '/home/amir/code/ltr_document_lvl_explainability/data/yahoo/original/set1.train.txt',
     'valid': '/home/amir/code/ltr_document_lvl_explainability/data/yahoo/original/set1.valid.txt',
     'test': '/home/amir/code/ltr_document_lvl_explainability/data/yahoo/original/set1.test.txt',
 },
  'mq2008':
    {
     'train': '/home/amir/code/ltr_document_lvl_explainability/data/mq2008/original/train.txt',
     'valid': '/home/amir/code/ltr_document_lvl_explainability/data/mq2008/original/vali.txt',
     'test': '/home/amir/code/ltr_document_lvl_explainability/data/mq2008/original/test.txt',  
    }
}

In [12]:
x_train, y_train, qid_train = load_svmlight_file(data_path[dataset_name]['train'], query_id=True)
x_valid, y_valid, qid_valid = load_svmlight_file(data_path[dataset_name]['valid'], query_id=True)
x_test, y_test, qid_test = load_svmlight_file(data_path[dataset_name]['test'], query_id=True)

def get_agg_count(qids):
    count = []
    unique_qids = np.unique(qids)
    for i in range(len(unique_qids)):
        count.append(len(np.argwhere(qids == unique_qids[i]).flatten()))
    return count

qid__train = get_agg_count(qid_train)
qid__valid = get_agg_count(qid_valid)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
y_valid = y_valid.astype(np.int32)

In [13]:
x_train.shape

(723412, 136)

In [51]:
len_docs = []
q_doc_idx = []

for q in np.unique(qid_test): 
    doc_idx = np.argwhere(qid_test == q).flatten()
    len_docs.append(len(doc_idx))

for q in np.unique(qid_test):
    doc_idx = np.argwhere(qid_test == q).flatten()
    if len(doc_idx) <= np.median(len_docs):
        q_doc_idx.append(q)

In [53]:
valid_len_docs = []
valid_q_doc_idx = []

for q in np.unique(qid_valid): 
    doc_idx = np.argwhere(qid_valid == q).flatten()
    valid_len_docs.append(len(doc_idx))

for q in np.unique(qid_valid):
    doc_idx = np.argwhere(qid_valid == q).flatten()
    if len(doc_idx) <= np.median(valid_len_docs):
        valid_q_doc_idx.append(q)

ranker = lightgbm.LGBMRanker(
                    objective="lambdarank",
                    boosting_type = "gbdt",
                    #n_estimators = 25,
                    importance_type = "gain",
                    metric= "ndcg",
                    #num_leaves = 10,
                    #learning_rate = 0.05,
                    #max_depth = -1,
                    label_gain =[i for i in range(max(y_train.max(), y_test.max()) + 1)])
                    
                    
ranker.fit(
      X=x_train,
      y=y_train,
      group=qid__train,
      eval_set=[(x_train, y_train),(x_valid, y_valid)],
      eval_group=[qid__train, qid__valid],
      eval_at=[5, 10])
      
ranker.booster_.save_model('./models/lmart_{}_v2.txt'.format(dataset_name))
pickle.dump(ranker.evals_result_, open( "./models/lmart_{}_eval_v2.p".format(dataset_name), "wb" ) )

In [55]:
unique_q_id_test = np.unique(qid_test)
data_sample = {}
for i in range(30):
    doc_idx = np.argwhere(q_doc_idx[i] == qid_test).flatten()
    data_sample[unique_q_id_test[i]] = x_test[doc_idx].toarray()

pickle.dump(data_sample, open( "./data/{}_test_sample_v2.p".format(dataset_name), "wb" ) )


unique_q_id_valid = np.unique(qid_valid)

data_background = {}
for i in range(50):
    doc_idx = np.argwhere(valid_q_doc_idx[i] == qid_valid).flatten()
    data_background[valid_q_doc_idx[i]] = x_valid[doc_idx].toarray()
pickle.dump(data_background, open( "./data/{}_background_v3.p".format(dataset_name), "wb" ) )

In [ ]:
ranker = lightgbm.Booster(model_file='./models/lmart_{}.txt'.format(dataset_name))
ranker.params['objective'] = 'binary'

background_dict = pickle.load( open( "./data/web10k_background.p", "rb" ) )
test = pickle.load( open( "./data/web10k_test_sample.p", "rb" ) )

background = []
for q in background_dict:
    background.extend(background_dict[q])
background = np.array(background).reshape(-1, test[13].shape[1])

lirme = LIRME(background)
e_sample_size = 4000

p_exps = {}
#exps = {'lime': [],  'shap': [], 'lirme': [], 'exs_v1': [],  'exs_v2': [], 'grad_d': [], 'random_d': []}
pointwise_exp = ['lime', 'shap', 'lirme', 'exs_v1', 'exs_v2', 'grad_d', 'random_d']

#for key in test.keys():
for key in [13]:
    p_exps[key] = {}
    for e in pointwise_exp:
        p_exps[key][e] = []
    print(p_exps)
    q_docs = test[key]
    pred_q_docs = ranker.predict(q_docs)
    print(q_docs.shape, pred_q_docs)
        
    #for i in range(len(q_docs)): 
    for i in range(2): 
        instance = q_docs[i]
        p_exps[key]['lime'].append(lime_exp(instance, ranker, background, sample_size=e_sample_size))
        p_exps[key]['shap'].append(shap_exp(instance, background, ranker, sample_size=e_sample_size))


        exp_lirme = lirme.explain(instance, ranker.predict, pred_q_docs, 
                    sur_type='ridge', label_type='regression', 
                    instance_idx=i, top_rank=5, sample_size=e_sample_size)
        p_exps[key]['lirme'].append(exp_lirme)

        gradient_exp = grad_exp(instance, ranker)
        p_exps[key]['grad_d'].append(gradient_exp)

        exp_exs_v1 = lirme.explain(instance, ranker.predict, pred_q_docs, 
                    sur_type='svm', label_type='top_k_binary', 
                    instance_idx=i, top_rank=5, sample_size=e_sample_size)
        p_exps[key]['exs_v1'].append(exp_exs_v1)

        exp_exs_v2 = lirme.explain(instance, ranker.predict, pred_q_docs, 
                    sur_type='svm', label_type='score', 
                    instance_idx=i, top_rank=5, sample_size=e_sample_size)
        p_exps[key]['exs_v2'].append(exp_exs_v2)

        p_exps[key]['random_d'].append(random_exp(instance.shape[0]))

pickle.dump(p_exps, open( "./web10k_pointwise_exps.p", "wb" ) )

In [5]:
pointwise_exp = pickle.load( open( "./exps/{}_pointwise_exps_v1.p".format(dataset_name), "rb" ) )

In [12]:
pointwise_exp[13].keys()

dict_keys(['lime', 'shap', 'lirme', 'exs_v1', 'exs_v2', 'grad_d', 'random_d'])

from listwise_exp import pmi_exp, rpi_exp, grad_q_exp, greedy_exp, random_q_exp
listwise_exp = ['pmi', 'rpi', 'grad', 'rank_lime', 'greedy_score', 'random']

from ranklime import RankLIME
rlime = RankLIME(background)

l_exps = {}

for key in [13]:
    doc_values = test[key]
    l_exps[key] = {}
    for e in listwise_exp:
        l_exps[key][e] = {}
    
    l_exps[key]['pmi'] = pmi_exp(doc_values, ranker)
    l_exps[key]['rpi'] = rpi_exp(doc_values, ranker)
    l_exps[key]['grad'] = grad_q_exp(doc_values, ranker)
    l_exps[key]['rank_lime'] = rlime.explain(doc_values, ranker.predict, [])
    l_exps[key]['greedy_score'] = greedy_exp(doc_values, ranker.predict)
    l_exps[key]['random'] = random_q_exp(doc_values)

pickle.dump(l_exps, open( "./exps/web10k_listwise_exps.p", "wb" ) )

In [8]:
listwise_exp = pickle.load( open( "./exps/{}_listwise_exps.p".format(dataset_name), "rb" ) )

In [11]:
listwise_exp[13].keys()

dict_keys(['pmi', 'rpi', 'grad', 'rank_lime', 'greedy_score', 'random'])